In [1]:
%matplotlib inline
from utils import *
from plotting import *
from models import *
import os
import pickle

## Results

In [6]:
tasks = ['doubleTmaze', 'cylinder', 'permanence', 'thorndike']
model_types = ['sparse-AE', 'AE']

In [2]:
results = {}
for model_type in model_types:
    results[model_type] = {}
    for task in tasks:
        print(model_type, task)
        
        # Load dataset and model
        dataset, position = load_dataset(directory='datasets/' + task, task=task)
        model = Conv_AE(n_hidden=1000).to('cuda')
        model.load_state_dict(torch.load('saved_models/main_{}_{}_1000hidden.pth'.format(task, model_type)))

        # Get latent embeddings
        embeddings = get_latent_vectors(dataset, model)
        embeddings[embeddings < 1e-5] = 0
        
        # Generate ratemaps and cleaned ratemaps
        ratemaps = generate_ratemaps(embeddings, position, n_bins=60, filter_width=4, n_bins_padding=10)
        cleaned_ratemaps = clean_ratemaps(ratemaps)
        
        # Place field density
        all_num_fields, all_centroids, all_sizes = stats_place_fields(cleaned_ratemaps, min_pix_cluster=0.03)
        place_field_density = np.histogram(all_num_fields, bins=np.max(all_num_fields) + 1, density=True)[0]
        
        # Homogeneity in tiling of 2D space
        homogeneity_tiling = homogeneity_2Dtiling(all_centroids)
        
        # Distribution of distances to walls
        occupancy_map = generate_occupancy_map(position, n_bins=60, filter_width=0, n_bins_padding=10, norm=False)
        distances = dist_to_walls(all_centroids, occupancy_map)
        dist_size_corr_slope, b = np.polyfit(distances, all_sizes, 1)

        # Average spatial information
        unfiltered_ratemaps = generate_ratemaps(embeddings, position, n_bins=30, filter_width=0, n_bins_padding=0)
        cleaned_unfiltered_ratemaps = clean_ratemaps(unfiltered_ratemaps)
        occupancy_map_norm = generate_occupancy_map(position, n_bins=30, filter_width=0, n_bins_padding=0, norm=True)
        avg_spatial_info = np.mean(spatial_information(cleaned_unfiltered_ratemaps, occupancy_map_norm))
    
        # Linear decoding error
        #linear_score, baseline_score, ratio = linear_decoding_score(embeddings, position, n_baseline=100)
        norm_factor = (np.max(position) - np.min(position)) * np.sqrt(2)   # uses the diagonal of the square as max distance.
        mean_decoding_error_embeddings = linear_decoding_error(embeddings, position, norm=norm_factor)
        
        ratemaps_reshaped = ratemaps.reshape(ratemaps.shape[1] * ratemaps.shape[2], ratemaps.shape[0])
        pos_values = np.linspace(np.min(position), np.max(position), ratemaps.shape[1])
        position_for_ratemaps = np.array([[i, j] for i in pos_values for j in pos_values])
        mean_decoding_error_ratemaps = linear_decoding_error(ratemaps_reshaped, position_for_ratemaps, norm=norm_factor)
        
        # Store the results in the dictionary
        results[model_type][task] = {
            'ratemaps': cleaned_ratemaps,
            'place_field_density_hist': place_field_density,
            'homogeneity_2Dtiling': homogeneity_tiling,
            'distances_to_walls': distances,
            'all_sizes': all_sizes,
            'dist_size_corr_slope': dist_size_corr_slope,
            'avg_spatial_info': avg_spatial_info,
            #'linear_decoding_score': linear_score,
            #'linear_decoding_baseline_score': baseline_score,
            #'linear_decoding_ratio': ratio,
            'mean_decoding_error_embeddings': mean_decoding_error_embeddings,
            'mean_decoding_error_ratemaps': mean_decoding_error_ratemaps
        }

sparse-AE doubleTmaze
sparse-AE cylinder
sparse-AE permanence
sparse-AE thorndike
AE doubleTmaze
AE cylinder
AE permanence
AE thorndike


#### Save results

In [3]:
# Ensure the directory exists
os.makedirs('results', exist_ok=True)

# Save the results dictionary to a file in the 'results' folder
file_path = os.path.join('results', 'spatial_allModels.pkl')
with open(file_path, 'wb') as f:
    pickle.dump(results, f)

#### Load results

In [3]:
# Load and print the results to check
file_path = os.path.join('results', 'spatial_allModels.pkl')
with open(file_path, 'rb') as f:
    results = pickle.load(f)

## Plots

In [4]:
results['AE']['cylinder'].keys()

dict_keys(['ratemaps', 'place_field_density_hist', 'homogeneity_2Dtiling', 'distances_to_walls', 'all_sizes', 'dist_size_corr_slope', 'avg_spatial_info', 'mean_decoding_error_embeddings', 'mean_decoding_error_ratemaps'])

In [7]:
plot_ratemaps(results[model_types[1]][tasks[1]]['ratemaps'])